# Recent Shocks Vulnerability Indicator

Notebook environment to migrate EM-DAT csv file to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [2]:
#%pip install tqdm

<IPython.core.display.Javascript object>

In [3]:
# Import standard packages
import os
import pathlib

import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import math
from tqdm import tqdm 

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
gca_data_dir = r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\03_Vulnerability"


# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)

<IPython.core.display.Javascript object>

In [4]:
# Project paths & files (manual input)
dataset_dir = pathlib.Path().joinpath(gca_data_dir,"Recent_Shocks")
dataset_dir_path = dataset_dir.joinpath("Recent_Shocks_original.nc")
CF_dir = dataset_dir.joinpath("CF")  # directory to save output CF check files
CF_dir

WindowsPath('P:/11209197-018-global-coastal-atlas/MSc_students/ClenmarRowe/Data/All_Datasets/Orig_Datasets/03_Vulnerability/Recent_Shocks/CF')

<IPython.core.display.Javascript object>

In [5]:
# Use A.Liundijk transects as template_base

df_transects = pd.read_csv(r'P:\1000545-054-globalbeaches\03_Global_shorelines_update_2020\Transect_Mapping\Version_16102018\CSV\ShorelineMonitor_1984_2021_v1.5_set1_filtered.csv')
df_transects.head(2)


,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt,dist,...,outliers_2,Timespan,intercept,intercept_unc,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat
0,BOX_028_183_0,CHL,South America,Chile,14.088576,1.503763,True,8.0,"[14.0, 16.0, 19.0, 20.0, 22.0, 30.0, 32.0, 35.0]","[242.54210813082503, 300.2922429940901, 346.20...",...,[6],21.0,56.678813,35.137902,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456
1,BOX_028_183_1,CHL,South America,Chile,6.937372,7.717642,False,13.0,"[0.0, 14.0, 16.0, 18.0, 19.0, 20.0, 22.0, 24.0...","[711.6071166056788, 331.617300061847, 351.8535...",...,[],21.0,297.416535,196.569308,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940


<IPython.core.display.Javascript object>

In [8]:

gdf_transects = gpd.GeoDataFrame(df_transects, geometry=gpd.points_from_xy(df_transects.Intersect_lon, df_transects.Intersect_lat)).set_crs('epsg:4326').to_crs('+proj=wintri')
print(len(gdf_transects))
gdf_transects.head(2)

1739826


,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt,dist,...,Timespan,intercept,intercept_unc,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,geometry
0,BOX_028_183_0,CHL,South America,Chile,14.088576,1.503763,True,8.0,"[14.0, 16.0, 19.0, 20.0, 22.0, 30.0, 32.0, 35.0]","[242.54210813082503, 300.2922429940901, 346.20...",...,21.0,56.678813,35.137902,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456,POINT (-7103378.783 -5764554.670)
1,BOX_028_183_1,CHL,South America,Chile,6.937372,7.717642,False,13.0,"[0.0, 14.0, 16.0, 18.0, 19.0, 20.0, 22.0, 24.0...","[711.6071166056788, 331.617300061847, 351.8535...",...,21.0,297.416535,196.569308,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940,POINT (-7102945.225 -5764704.629)


<IPython.core.display.Javascript object>

In [9]:
gdf_transects.columns


Index(['transect_id', 'country_id', 'continent', 'country_name', 'changerate',
       'changerate_unc', 'flag_sandy', 'no_shorelines', 'dt', 'dist', 'RMSE',
       'outliers_1', 'outliers_2', 'Timespan', 'intercept', 'intercept_unc',
       'Start_lon', 'Start_lat', 'Intersect_lon', 'Intersect_lat', 'End_lon',
       'End_lat', 'geometry'],
      dtype='object')

<IPython.core.display.Javascript object>

In [16]:
gdf_transects['transect_id'].dtype

dtype('O')

<IPython.core.display.Javascript object>

In [17]:
gdf_transects=gdf_transects.drop(columns=gdf_transects.columns[4:16])
gdf_transects


,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat,geometry
0,BOX_028_183_0,CHL,South America,Chile,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456,POINT (-7103378.783 -5764554.670)
1,BOX_028_183_1,CHL,South America,Chile,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940,POINT (-7102945.225 -5764704.629)
2,BOX_028_183_2,CHL,South America,Chile,-74.378628,-50.380629,-74.383284,-50.385527,-74.387941,-50.390425,POINT (-7102511.675 -5764854.586)
3,BOX_028_183_3,CHL,South America,Chile,-74.373950,-50.382583,-74.379517,-50.387079,-74.385083,-50.391574,POINT (-7102081.743 -5765012.406)
4,BOX_028_183_4,CHL,South America,Chile,-74.370425,-50.384358,-74.375991,-50.388853,-74.381558,-50.393348,POINT (-7101663.767 -5765196.247)
...,...,...,...,...,...,...,...,...,...,...,...
1739821,BOX_211_067_149,RUS,Europe,Russia,39.929937,64.701462,39.935198,64.698350,39.940460,64.695238,POINT (3399292.132 7245048.788)
1739822,BOX_211_067_150,RUS,Europe,Russia,39.933577,64.702586,39.938839,64.699474,39.944100,64.696363,POINT (3399560.443 7245181.237)
1739823,BOX_211_067_151,RUS,Europe,Russia,39.935546,64.703502,39.942003,64.700833,39.948460,64.698164,POINT (3399780.182 7245338.675)
1739824,BOX_211_067_152,RUS,Europe,Russia,39.937050,64.704370,39.944697,64.702356,39.952343,64.700341,POINT (3399954.257 7245513.190)


<IPython.core.display.Javascript object>

In [ ]:

df_transects=df_transects.drop(columns=df_transects.columns[4:16])
df_transects

,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat
0,BOX_028_183_0,CHL,South America,Chile,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456
1,BOX_028_183_1,CHL,South America,Chile,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940
2,BOX_028_183_2,CHL,South America,Chile,-74.378628,-50.380629,-74.383284,-50.385527,-74.387941,-50.390425
3,BOX_028_183_3,CHL,South America,Chile,-74.373950,-50.382583,-74.379517,-50.387079,-74.385083,-50.391574
4,BOX_028_183_4,CHL,South America,Chile,-74.370425,-50.384358,-74.375991,-50.388853,-74.381558,-50.393348
...,...,...,...,...,...,...,...,...,...,...
1739821,BOX_211_067_149,RUS,Europe,Russia,39.929937,64.701462,39.935198,64.698350,39.940460,64.695238
1739822,BOX_211_067_150,RUS,Europe,Russia,39.933577,64.702586,39.938839,64.699474,39.944100,64.696363
1739823,BOX_211_067_151,RUS,Europe,Russia,39.935546,64.703502,39.942003,64.700833,39.948460,64.698164
1739824,BOX_211_067_152,RUS,Europe,Russia,39.937050,64.704370,39.944697,64.702356,39.952343,64.700341


<IPython.core.display.Javascript object>

In [ ]:
#Save Arjens clocation template in folder
df_transects.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\04_Auxillary_files\Arjen_Vector_Template",index=False)

<IPython.core.display.Javascript object>

In [ ]:
df_template=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\04_Auxillary_files\Arjen_Vector_Template")
df_template

,transect_id,country_id,continent,country_name,Start_lon,Start_lat,Intersect_lon,Intersect_lat,End_lon,End_lat
0,BOX_028_183_0,CHL,South America,Chile,-74.386310,-50.377659,-74.390966,-50.382558,-74.395623,-50.387456
1,BOX_028_183_1,CHL,South America,Chile,-74.382469,-50.379144,-74.387125,-50.384042,-74.391782,-50.388940
2,BOX_028_183_2,CHL,South America,Chile,-74.378628,-50.380629,-74.383284,-50.385527,-74.387941,-50.390425
3,BOX_028_183_3,CHL,South America,Chile,-74.373950,-50.382583,-74.379517,-50.387079,-74.385083,-50.391574
4,BOX_028_183_4,CHL,South America,Chile,-74.370425,-50.384358,-74.375991,-50.388853,-74.381558,-50.393348
...,...,...,...,...,...,...,...,...,...,...
1739821,BOX_211_067_149,RUS,Europe,Russia,39.929937,64.701462,39.935198,64.698350,39.940460,64.695238
1739822,BOX_211_067_150,RUS,Europe,Russia,39.933577,64.702586,39.938839,64.699474,39.944100,64.696363
1739823,BOX_211_067_151,RUS,Europe,Russia,39.935546,64.703502,39.942003,64.700833,39.948460,64.698164
1739824,BOX_211_067_152,RUS,Europe,Russia,39.937050,64.704370,39.944697,64.702356,39.952343,64.700341


<IPython.core.display.Javascript object>